# **Cài đặt môi trường**

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [38]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [39]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# Load model bloomz-1b1
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to("cuda")
# Load tokenizer từ bloomz-1b1
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

# **Tải dataset viquad**

In [10]:
!wget https://raw.githubusercontent.com/phamdinhkhanh/Vietnamese_LLMs/main/Generate_and_Translate_Dataset/Vietnamese_Instructions_datasets/Translation/Alpaca_52k/NLLB_1B3_results/NLLB_translations_Vietnamese_0_10k.json

--2023-06-24 11:18:10--  https://raw.githubusercontent.com/phamdinhkhanh/Vietnamese_LLMs/main/Generate_and_Translate_Dataset/Vietnamese_Instructions_datasets/Translation/Alpaca_52k/NLLB_1B3_results/NLLB_translations_Vietnamese_0_10k.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9623917 (9.2M) [text/plain]
Saving to: ‘NLLB_translations_Vietnamese_0_10k.json’

NLLB_translations_V 100%[===================>]   9.18M  --.-KB/s    in 0.04s   

2023-06-24 11:18:10 (249 MB/s) - ‘NLLB_translations_Vietnamese_0_10k.json’ saved [9623917/9623917]



In [63]:
from datasets import Dataset
ds= Dataset.from_json("NLLB_translations_Vietnamese_0_10k.json")

In [64]:
ds

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 10000
})

In [65]:
ds[0]

{'input': '',
 'instruction': 'Hãy cho tôi ba mẹo để giữ sức khỏe.',
 'output': '1 Ăn một chế độ ăn uống cân bằng và bổ dưỡng Hãy chắc chắn rằng bữa ăn của bạn bao gồm nhiều loại trái cây và rau quả, protein nạc, ngũ cốc nguyên hạt và chất béo lành mạnh Điều này giúp cung cấp cho cơ thể các chất dinh dưỡng thiết yếu để hoạt động tốt nhất và có thể giúp ngăn ngừa các bệnh mãn tính 2 Tập thể dục thường xuyên Tập thể dục rất quan trọng để duy trì xương khỏe mạnh cơ bắp và sức khỏe tim mạch Mục tiêu ít nhất 150 phút tập thể dục aerobic vừa phải hoặc 75 phút tập thể dục mạnh mỗi tuần 3 Hãy ngủ đủ giấc ngủ Giấc ngủ đủ chất lượng rất quan trọng cho sức khỏe thể chất và tinh thần Nó giúp điều chỉnh tâm trạng cải thiện chức năng nhận thức và hỗ trợ tăng trưởng và chức năng miễn dịch khỏe mạnh Mục tiêu ngủ 7 9 giờ mỗi đêm '}

In [66]:
from transformers import pipeline, set_seed, GPT2LMHeadModel, AutoTokenizer, TextDataset, AutoModelForCausalLM
import datasets
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer,IntervalStrategy, EarlyStoppingCallback
import numpy as np
import math

from datasets import load_dataset

In [67]:
# Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# phía sau
def merge_columns(example):
  # "|||"+example["context"]+"|||:"
    example["prediction"] =  "Hỏi:"+ example["instruction"] + '''
    Đáp: ''' + str(example["output"])
    return example

In [68]:
# Map function trên vào dataset
data = ds.map(merge_columns)

In [69]:
# Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
def tokeni(example):
    example["prediction"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
    return example

In [70]:
# Map function trên vào dataset
tokenized_datasets = data.map(
    tokeni,
    batched=True
)


In [71]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [72]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1179648 || all params: 1066493952 || trainable%: 0.11060990995662018


In [56]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets["prediction"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,4.594600
2,3.388200
3,3.374700
4,2.976800
5,4.123600
6,2.353900
7,4.616600
8,2.534100
9,4.087800
10,4.119500


TrainOutput(global_step=200, training_loss=2.721413468122482, metrics={'train_runtime': 124.9776, 'train_samples_per_second': 1.6, 'train_steps_per_second': 1.6, 'total_flos': 678593437618176.0, 'train_loss': 2.721413468122482, 'epoch': 0.02})

In [57]:
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [58]:
print(ds["instruction"][0])

Hãy cho tôi ba mẹo để giữ sức khỏe.


In [59]:
question = ds["instruction"][0]
prompt = "Hỏi:"+ question + '''
    Đáp:'''
prompt

'Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.\n    Đáp:'

In [79]:
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=3,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: Ăn uống điều độ.
Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: Ăn uống điều độ, ngủ đủ giấc, tập thể dục đều đặn.
Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: Ăn uống điều độ, ngủ đủ giấc, và tập thể dục đều đặn.


In [83]:
question2 = "Ai là họa sĩ nổi tiếng nhất thế giới?"
prompt = "Hỏi:"+ question2 + '''
    Đáp:'''
prompt
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=3,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Ai là họa sĩ nổi tiếng nhất thế giới?
    Đáp: Van Gogh
Hỏi:Ai là họa sĩ nổi tiếng nhất thế giới?
    Đáp: Picasso
Hỏi:Ai là họa sĩ nổi tiếng nhất thế giới?
    Đáp: Matisse
